# A whirlwind tour through the Microsoft Semantic Kernel

In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planning.basic_planner import BasicPlanner

load_dotenv()
OPENAI_ENDPOINT = os.getenv("OPENAI_USEAST3_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_USEAST3_API_KEY")


## Initializing the Kernel

A cool thing about Semantic Kernel is that it supports multiple models. This enables you to run simple workloads on cheaper models, and expensive workloads on expensive models.

In [2]:
kernel = sk.Kernel()

gpt35 = AzureChatCompletion(deployment_name="gpt-35-turbo", # yours may be different
                            endpoint=OPENAI_ENDPOINT,
                            api_key=OPENAI_API_KEY)

gpt4 = AzureChatCompletion(deployment_name="gpt-4", # yours may be different
                            endpoint=OPENAI_ENDPOINT,
                            api_key=OPENAI_API_KEY)

kernel.add_chat_service("gpt35", gpt35)
kernel.add_chat_service("gpt4", gpt4)

## Calling the chat service

One of the simplest things you can do is simply execute a prompt.

In [3]:
prompt = """knock, knock? Who’s there? {{$input}}. {{$input}} who?"""
knock = kernel.create_semantic_function(prompt, temperature=0.8)
response = knock("Dishes")
print(response)


Dishes the police, open up!


## Semantic functions

A **semantic function** is a function that interacts with a large language model (LLM).

Although you can define a semantic function with a `dict` for the configuration and a `string` for the prompt as we did above, in production, we usually prefer to separate the code from the prompts and configuration. 

### Plugins

Collections of semantic functions are called **Plugins**. Plugins are simply folders that contain semantic functions. Each semantic function should be in a separate folder.

Each semantic function is defined by two files : `skprompt.txt` that contains the prompt (including placeholders for parameters) and `config.json`, that contains the configuration, such as default *temperature*, default *service*, etc.



In [4]:
jokes_plugin = kernel.import_semantic_skill_from_directory("plugins", "jokes")
jokes_plugin

{'cross_the_road_joke': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ce6e6520d0>,
 'genie_joke': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ce7d72aa10>,
 'knock_knock_joke': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1ce7d72b390>}

### Running a function from a plugin

Once you loaded the functions into the Kernel, you can load them into a varaible or simply accessing them directly from the plugin object.

In [5]:
response = jokes_plugin["knock_knock_joke"]("dishes")
print(response)

Knock, knock?
Who's there?
Dishes.
Dishes who?
Dishes your friend, open the door and let me in!


Here we load the function into a function variable.

In [6]:
cross_the_road = jokes_plugin["cross_the_road_joke"]
response = cross_the_road("atom")
print(response)

Why did the atom cross the road?

Because it wanted to split!


This `genie_joke` function has multiple parameters. To pass them, you have to create an object of the `ContextVariables` class, and pass it to the function.

In [7]:
context_variables = sk.ContextVariables()
context_variables["firstWish"] = "go home"
context_variables["secondWish"] = "go home"

response = jokes_plugin["genie_joke"](variables=context_variables)
print(response)

Three men were on a deserted island and found a genie. The genie gave each one wish. 

The first said he wished to go home and instantly transformed into a homing pigeon.

The second said he wished to go home and in a puff of smoke, he turned into a houseplant in his living room.

The third person’s wish was to have both his friends back on the island because he was getting bored.


## Native functions

Native functions are pure Python code. We don't really need to have them in the same directory as the semantic function, but I like doing that because it makes it easier to find them. We import them using the `import_skill` method.

The function below classifies an image.

In [8]:
from plugins.image_classifier_plugin.image_classifier import ImageClassifierPlugin
image_classifier = ImageClassifierPlugin()
classify_plugin = kernel.import_skill(image_classifier, skill_name="classify_image")

This is just a silly list of URLs to test the image classifier function.

In [9]:
url = "https://cdn.pixabay.com/photo/2016/02/10/16/37/cat-1192026_1280.jpg"
url = "https://mpsocial.blob.core.windows.net/blog-images/fuzzychair.png"
url = "https://mpsocial.blob.core.windows.net/blog-images/fail-whale.webp"
url = "https://mpsocial.blob.core.windows.net/blog-images/rat.jpeg"

# other pictures to try: http://fun-pictube.blogspot.com/2012/05/animal-pictures-zoo-animal-pictures.html
url = "http://3.bp.blogspot.com/-fZK39AQB37M/T6Z1104yXWI/AAAAAAAAGko/c3Sv77URwPk/s1600/animal+pictures+%25285%2529.jpg"
url = "http://2.bp.blogspot.com/-tG6z7DOsHNc/T6Z1DuzXs9I/AAAAAAAAGfY/YTmFDxw0Qxg/s320/animal+pictures+%252812%2529.jpg"



In [10]:
response = classify_plugin["classify_image"](url)

# get only up to the first comma, if it exists
print(response)

tiger


## Calling multiple functions in sequence

The big advantage of using a Kernel is that you can call multiple functions in sequence, and pass the output of one function to the next one. This allows you to do complex workflows in a simple call. 

### Telling a joke about an image

In the example below, we pass an image as a URL, and then call the `classify_image` native function and the `cross_the_road` semantic function in sequence.

In [11]:
context = kernel.create_new_context()
context["input"] = url

response = await kernel.run_async(
    classify_plugin["classify_image"],
    jokes_plugin["cross_the_road_joke"],
    input_context=context
)

print(response)

Why did the tiger cross the road?

Because it was the chicken's day off.


## Planner

The planner allows you to create an ask in natural language. The Semantic Kernel will search the plugins for a list of functions that matches the ask, and then execute them in sequence.

In [12]:
planner = BasicPlanner()
ask = f"""Write a cross the road joke after classifying the image with this url: {url}."""

In [13]:
plan = await planner.create_plan_async(ask, kernel)
print(plan.generated_plan)

{
    "input": "http://2.bp.blogspot.com/-tG6z7DOsHNc/T6Z1DuzXs9I/AAAAAAAAGfY/YTmFDxw0Qxg/s320/animal+pictures+%252812%2529.jpg",
    "subtasks": [
        {"function": "classify_image.classify_image"},
        {"function": "jokes.cross_the_road_joke"}
    ]
}


In [14]:
joke_from_image = await planner.execute_plan_async(plan, kernel)
print(joke_from_image)

Why did the tiger cross the road?

Because it wanted to show the chicken that it was possible!
